In [ ]:
# http://boell-sammlungen.faust-web.de/szeig.FAU?sid=DA10D13E22&dm=1&erg=A&qpos=12&rpos=11&ipos=11
# http://boell-sammlungen.faust-web.de/zvimg.FAU?sid=DA10D13E&DM=1&qpos=12062&ipos=1&erg=A&hst=1&rpos=sammlungen.jpg

In [ ]:
# http://boell-sammlungen.faust-web.de/

In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import shutil
import os

In [ ]:
url_start='http://boell-sammlungen.faust-web.de/szeig.FAU?sid='
ssid=str(input("Insert session id: "))
dm= '&dm='
num_dm=1
erg='&erg=A'
qpos= '&qpos='
num_qpos=3
rpos='&rpos='
num_rpos= 2
ipos='&ipos='
num_ipos=2

final_urls=[]
num_page= int(input("Insert number of pages: "))
for search in range(1,num_page):
    final_search=str(url_start+ssid+dm+str(num_dm)+erg+qpos+str(num_qpos)+rpos+str(num_rpos)+ipos+str(num_ipos))
    num_qpos+=1
    num_rpos+=1
    num_ipos+=1
    reqs = requests.get(final_search)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    txt= '.*.jpg'
    for url in re.findall(txt,str(soup)):
        url=url.replace('<a href="','http://boell-sammlungen.faust-web.de/')
        final_urls.append(url)

final_urls = [x for x in final_urls if not x.startswith('imgkh1')]
final_urls= list(set(final_urls))
len(final_urls)

final_images=[]

for link in final_urls:
    reqs = requests.get(link)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    for url in soup.find_all('img'):
        url=url['src']
        final_images.append(url)

final_images = [x for x in final_images if not x.startswith('data:')]

boel='http://boell-sammlungen.faust-web.de/'
final_images = [boel+x for x in final_images]

print("Image URLs extracted...")
final_df= pd.DataFrame()
final_df['site_images']= final_images
final_df= final_df.drop_duplicates()
search_term= str(input("search term: "))
csv_name= search_term+"_boell_sammlungen_image_urls"+".csv"
final_df.to_csv(csv_name,index=False)
final_images= final_df['site_images'].to_list()

search_term= search_term.replace(' ','')
search_term=search_term.replace(',','')
os.mkdir(os.path.join(os.getcwd(),"images_from_the_past",'boell_sammlungen',search_term))

img_num=0
for url in final_images:
    try:
        img_num+=1
        image_url = url
        filename= os.path.join(os.getcwd(),"images_from_the_past",'boell_sammlungen',search_term,str(img_num)+'.jpg')
        r = requests.get(image_url, stream = True)

        # Check if the image was retrieved successfully
        if r.status_code == 200:
            # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
            r.raw.decode_content = True

            # Open a local file with wb ( write binary ) permission.
            with open(filename,'wb') as f:
                shutil.copyfileobj(r.raw, f)
        else:
            print('Image Couldn\'t be retreived')
    except:
        continue